In [1]:
import json
import time
from flask import Flask, jsonify, request, Response
import requests
import sys
from datetime import datetime

def post_json_request(url, obj):
    return requests.post(url, json=obj).json()

In [2]:
database="test10"
project="adc_cali"
start_time = time.time()
pubmedResult = post_json_request(
    "http://localhost:5000/query",
    {
        "database": database,
        "project": project,
        "maxdocs": 50,
        "patternid": 1,
        "query": "breast[Title/Abstract] AND cancer[Title/Abstract]",
    },
)[0]["exit"]
print("--- %s seconds ---" % (time.time() - start_time))

--- 242.7404510974884 seconds ---


In [3]:
print(pubmedResult)

0


In [4]:
metadata_documents = post_json_request(
    "http://localhost:5001/mongo-doc-list",
    {
        "db_name": database,
        "coll_name": f"{project}_metadata_global"
    },
)
metadata_documents[0].keys()

dict_keys(['_id', 'title', 'abstract', 'authors', 'dbid', 'doi', 'lang', 'org', 'pat_id', 'url', 'year'])

In [5]:
authors_documents = post_json_request(
    "http://localhost:5001/mongo-doc-list",
    {
        "db_name": database,
        "coll_name": f"{project}_author_vs_doc_id_global"
    },
)
authors_documents[0].keys()

dict_keys(['_id', 'author', 'doc_id', 'doi'])

In [6]:
urlList=list(map(lambda document: {"url": document["url"]}, metadata_documents))
urlList

[{'url': None},
 {'url': 'https://link.springer.com/content/pdf/10.1007/s00018-023-04734-7.pdf'},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': 'http://www.plosone.org/article/fetchObject.action?uri=info:doi/10.1371/journal.pcbi.1010939&representation=PDF'},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': 'https://link.springer.com/content/pdf/10.1007/s10549-022-06815-w.pdf'},
 {'url': None},
 {'url': None},
 {'url': None},
 {'url': 'http://www.thelancet.com/pdfs/journals/lancet/PIIS0140-6736(22)02249-8.pdf'},
 {'url': 'http://www.thelancet.com/pdfs/journals/lancet/PIIS0140-6736(22)02241-3.pdf'},
 {'url': None},
 {'url':

In [7]:
availablePDFs=len(list([url_dict["url"]] for url_dict in urlList if url_dict["url"]))/len(urlList)*100
print(f"avaliable pdfs: {availablePDFs}%")

avaliable pdfs: 14.000000000000002%


In [8]:
def url2head(url):
    try:
        return post_json_request(
            "http://localhost:5002/url2htext",
            { "url": url }
        )
    except:
        return {"htext": ""}
url2head("http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9983066&blobtype=pdf")

{'htext': 'MEDICINE INTERNATIONAL 3: 11, 2023\n\nThe combination of positive anti‑WDR1 antibodies with\nnegative anti‑CFL1 antibodies in serum is a poor prognostic\nfactor for patients with esophageal carcinoma\nMASAAKI ITO1, SATOSHI YAJIMA2, TAKASHI SUZUKI2, YOKO OSHIMA2, TATSUKI NANAMI2,\nMAKOTO SUMAZAKI2, FUMIAKI SHIRATORI2, HAO WANG3,4, LIUBING HU4, HIROTAKA TAKIZAWA5,\nSHU‑YANG LI6, YASUO IWADATE6, TAKAKI HIWASA1,6 and HIDEAKI SHIMADA1,2\n1\n\nDepartment of Clinical Oncology, Toho University Graduate School of Medicine; 2Department of Gastroenterological Surgery,\nToho University School of Medicine, Tokyo 143‑8541, Japan; 3Stroke Center, The First Affiliated Hospital, Jinan University;\n4\nDepartment of Anesthesiology, Stroke Center, The First Affiliated Hospital and Health Science Center, Jinan University,\nGuangzhou, Guangdong 510630, P.R. China; 5Port Square Kashiwado Clinic, Kashiwado Memorial Foundation,\nChiba 260‑0025; 6Department of Neurological Surgery, Graduate School of

In [9]:
urls = list(filter(lambda url: url["url"] != None, urlList))
urls

[{'url': 'https://link.springer.com/content/pdf/10.1007/s00018-023-04734-7.pdf'},
 {'url': 'http://www.plosone.org/article/fetchObject.action?uri=info:doi/10.1371/journal.pcbi.1010939&representation=PDF'},
 {'url': 'https://link.springer.com/content/pdf/10.1007/s10549-022-06815-w.pdf'},
 {'url': 'http://www.thelancet.com/pdfs/journals/lancet/PIIS0140-6736(22)02249-8.pdf'},
 {'url': 'http://www.thelancet.com/pdfs/journals/lancet/PIIS0140-6736(22)02241-3.pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC10020071&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC10020396&blobtype=pdf'}]

In [10]:
start_time = time.time()
headers = list(map(lambda url: url2head(url["url"])["htext"], urls))
print("--- %s seconds ---" % (time.time() - start_time))
headers

--- 59.59257197380066 seconds ---


['Cellular and Molecular Life Sciences\n(2023) 80:100\nhttps://doi.org/10.1007/s00018-023-04734-7\n\nCellular and Molecular Life Sciences\n\nORIGINAL ARTICLE\n\nLoss of Kmt2c in vivo leads to EMT, mitochondrial dysfunction\nand improved response to lapatinib in breast cancer\nNikiana Simigdala1 · Anna Chalari1 · Aimilia D. Sklirou2 · Evangelia Chavdoula1,3,4 · George Papafotiou1 ·\nPelagia Melissa1 · Aimilia Kafalidou1 · Nikolaos Paschalidis1 · Ioannis S. Pateras5 · Emmanouil Athanasiadis1 ·\nDimitris Konstantopoulos6 · Ioannis P. Trougakos2 · Apostolos Klinakis1\nReceived: 4 May 2022 / Revised: 22 January 2023 / Accepted: 22 February 2023\n© The Author(s) 2023\n\nAbstract\nDeep sequencing of human tumours has uncovered a previously unappreciated role for epigenetic regulators in tumorigenesis.\nH3K4 methyltransferase KMT2C/MLL3 is mutated in several solid malignancies, including more than 10% of breast tumours.\nTo study the tumour suppressor role of KMT2C in breast cancer, we generat

In [11]:
def text2locations(text):
    try:
        return post_json_request(
            "http://localhost:5002/text2locations",
            { "text": text }
        )
    except:
        return {"locations": []}
text2locations("Exploration of Targeted Anti-tumor Therapy\nOpen Access Review\n\nAn overview of the anti-cancer actions of Tanshinones, derived\nfrom Salvia miltiorrhiza (Danshen)\nIrum Naz1, Myriam Merarchi2, Shanaya Ramchandani3, Muhammad Rashid Khan4*, Muhammad Nouman\nMalik1, Sumaira Sarwar1, Acharan S Narula5, Kwang Seok Ahn6*\nDepartment of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam University, Islamabad 45320, Pakistan\n\n1\n\nFaculty of Pharmacy, University of Paris Descartes, 75006 Paris, France\n\n2\n\nDepartment of Pharmacology-Biomedicine, The University of Melbourne, Parkville, VIC 3010, Australia\n\n3\n\nHigher Education Commission of Pakistan, Islamabad 44000, Pakistan\n\n4\n\nNarula Research, Chapel Hill, NC 27516, USA\n\n5\n\nDepartment of Science in Korean Medicine, College of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro,\nDongdaemun-gu, Seoul 02447, South Korea\n6\n\n*Correspondence: Muhammad Rashid Khan, Department of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam\nUniversity, Islamabad 45320, Pakistan. mrkhanqau@yahoo.com; Kwang Seok Ahn, Department of Science in Korean Medicine,\nCollege of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro, Dongdaemun-gu, Seoul 02447, South Korea. ksahn@\nkhu.ac.kr\nAcademic Editor: Gautam Sethi, National University of Singapore, Singapore\nReceived: April 16, 2020 Accepted: May 17, 2020 Published: June 29, 2020\n\nCite this article: Naz I, Merarchi M, Ramchandani S, Khan MR, Malik MN, Sarwar S, et al. An overview of the anti-cancer\nactions of Tanshinones, derived from Salvia miltiorrhiza (Danshen). Explor Target Antitumor Ther. 2020;1:153-70. https://doi.\norg/10.37349/etat.2020.00010\n\n")

{'locations': ['Tanshinones',
  'Salvia',
  'Biochemistry',
  'Pakistan',
  'Pharmacy',
  'Paris Descartes',
  'Paris',
  'France',
  'Melbourne',
  'Parkville',
  'Australia',
  'Pakistan Pakistan',
  'Science',
  'Korean Medicine',
  'Korean Medicine Seoul',
  'South',
  'Biochemistry Pakistan',
  'Science Korean Medicine',
  'Korean Medicine Seoul South Korea',
  'Singapore',
  'Tanshinones Salvia']}

In [12]:
start_time = time.time()
locations = list(map(lambda text: text2locations(text)["locations"], headers))
print("--- %s seconds ---" % (time.time() - start_time))
locatio

--- 0.9742515087127686 seconds ---


[['Cellular',
  'Author',
  'Athens',
  'Athens Greece',
  'Biophysics',
  'Biology',
  'Athens Athens',
  'Greece',
  'Cancer',
  'Genetics',
  'Columbus',
  'Columbus Pathology',
  'Athens Athens Greece'],
 ['Bayesian',
  'Mathematics',
  'Hong Kong',
  'Hong Kong China',
  'Mathematics Shenzhen',
  'China',
  'Life',
  'Northeastern University',
  'China China',
  'Chengdu',
  'China Ching',
  'Bayesian UNITED STATES',
  'China Bayesian'],
 ['Author',
  'New Zealand',
  'Epidemiology',
  'Biostatistics',
  'Auckland',
  'Auckland New',
  'Oncology',
  'Oncology Hamilton',
  'New',
  'NIDEA',
  'Demographic',
  'Waikato',
  'Hamilton'],
 ['Embase',
  'Nursing',
  'English',
  'Canada',
  'Australia',
  'Lebanon',
  'England',
  'Dublin',
  'Ireland',
  'Dublin Ireland',
  'Ireland Irish'],
 ['England',
  'Scotland',
  'Mean',
  'London',
  'Nutrition',
  'Health',
  'Medicine',
  'Brazil',
  'Cancer',
  'France'],
 ['Cancer',
  'U.S.',
  'US',
  'Guam',
  'Guam Hawai',
  'Guam Filipi

In [13]:
def text2places(text):
    try:
        return post_json_request(
            "http://localhost:5002/text2places",
            { "text": text }
        )
    except:
        return {"places": []}
text2places("Exploration of Targeted Anti-tumor Therapy\nOpen Access Review\n\nAn overview of the anti-cancer actions of Tanshinones, derived\nfrom Salvia miltiorrhiza (Danshen)\nIrum Naz1, Myriam Merarchi2, Shanaya Ramchandani3, Muhammad Rashid Khan4*, Muhammad Nouman\nMalik1, Sumaira Sarwar1, Acharan S Narula5, Kwang Seok Ahn6*\nDepartment of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam University, Islamabad 45320, Pakistan\n\n1\n\nFaculty of Pharmacy, University of Paris Descartes, 75006 Paris, France\n\n2\n\nDepartment of Pharmacology-Biomedicine, The University of Melbourne, Parkville, VIC 3010, Australia\n\n3\n\nHigher Education Commission of Pakistan, Islamabad 44000, Pakistan\n\n4\n\nNarula Research, Chapel Hill, NC 27516, USA\n\n5\n\nDepartment of Science in Korean Medicine, College of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro,\nDongdaemun-gu, Seoul 02447, South Korea\n6\n\n*Correspondence: Muhammad Rashid Khan, Department of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam\nUniversity, Islamabad 45320, Pakistan. mrkhanqau@yahoo.com; Kwang Seok Ahn, Department of Science in Korean Medicine,\nCollege of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro, Dongdaemun-gu, Seoul 02447, South Korea. ksahn@\nkhu.ac.kr\nAcademic Editor: Gautam Sethi, National University of Singapore, Singapore\nReceived: April 16, 2020 Accepted: May 17, 2020 Published: June 29, 2020\n\nCite this article: Naz I, Merarchi M, Ramchandani S, Khan MR, Malik MN, Sarwar S, et al. An overview of the anti-cancer\nactions of Tanshinones, derived from Salvia miltiorrhiza (Danshen). Explor Target Antitumor Ther. 2020;1:153-70. https://doi.\norg/10.37349/etat.2020.00010\n\n")

{'places': ['France',
  'Pakistan',
  'South Korea',
  'Singapore',
  'Australia',
  'United States of America',
  'Canada',
  'India',
  'Sweden',
  'New Zealand',
  'Denmark',
  'Indonesia']}

In [14]:
start_time = time.time()
places = list(map(lambda text: text2places(text)["places"], headers))
print("--- %s seconds ---" % (time.time() - start_time))
places

--- 38.82882833480835 seconds ---


[['Greece', 'United States of America', 'Canada', 'Spain'],
 ["People's Republic of China",
  'United States of America',
  'Japan',
  'India',
  'Mexico'],
 ['New Zealand',
  'Denmark',
  'Canada',
  'United States of America',
  'United Kingdom',
  'Australia',
  'Spain',
  'Kazakhstan',
  'France',
  'India'],
 ['Lebanon',
  'Canada',
  'Australia',
  'Vietnam',
  'Ireland',
  'United States of America',
  'Belarus',
  'Netherlands',
  'Ukraine'],
 ['France',
  'Cameroon',
  'Brazil',
  'United Kingdom',
  'Canada',
  'United States of America',
  'Serbia',
  'Spain',
  'Hungary',
  'Portugal',
  'Indonesia',
  'Ukraine'],
 ['United States of America', 'India', 'Spain', 'Japan'],
 ['Argentina']]

In [15]:
def text2emails(text):
    try:
        return post_json_request(
            "http://localhost:5002/text2emails",
            { "text": text }
        )
    except:
        return {"emails": []}
text2emails("Exploration of Targeted Anti-tumor Therapy\nOpen Access Review\n\nAn overview of the anti-cancer actions of Tanshinones, derived\nfrom Salvia miltiorrhiza (Danshen)\nIrum Naz1, Myriam Merarchi2, Shanaya Ramchandani3, Muhammad Rashid Khan4*, Muhammad Nouman\nMalik1, Sumaira Sarwar1, Acharan S Narula5, Kwang Seok Ahn6*\nDepartment of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam University, Islamabad 45320, Pakistan\n\n1\n\nFaculty of Pharmacy, University of Paris Descartes, 75006 Paris, France\n\n2\n\nDepartment of Pharmacology-Biomedicine, The University of Melbourne, Parkville, VIC 3010, Australia\n\n3\n\nHigher Education Commission of Pakistan, Islamabad 44000, Pakistan\n\n4\n\nNarula Research, Chapel Hill, NC 27516, USA\n\n5\n\nDepartment of Science in Korean Medicine, College of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro,\nDongdaemun-gu, Seoul 02447, South Korea\n6\n\n*Correspondence: Muhammad Rashid Khan, Department of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam\nUniversity, Islamabad 45320, Pakistan. mrkhanqau@yahoo.com; Kwang Seok Ahn, Department of Science in Korean Medicine,\nCollege of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro, Dongdaemun-gu, Seoul 02447, South Korea. ksahn@\nkhu.ac.kr\nAcademic Editor: Gautam Sethi, National University of Singapore, Singapore\nReceived: April 16, 2020 Accepted: May 17, 2020 Published: June 29, 2020\n\nCite this article: Naz I, Merarchi M, Ramchandani S, Khan MR, Malik MN, Sarwar S, et al. An overview of the anti-cancer\nactions of Tanshinones, derived from Salvia miltiorrhiza (Danshen). Explor Target Antitumor Ther. 2020;1:153-70. https://doi.\norg/10.37349/etat.2020.00010\n\n")

{'emails': ['mrkhanqau@yahoo.com']}

In [16]:
start_time = time.time()
emails = list(map(lambda text: text2emails(text)["emails"], headers))
print("--- %s seconds ---" % (time.time() - start_time))
emails

--- 0.04236745834350586 seconds ---


[['nsimigdala@bioacademy.gr',
  'konstantopoulos@fleming.gr',
  'aklinakis@bioacademy.gr',
  'itrougakos@biol.uoa.gr',
  'achalari@bioacademy.gr',
  'asklirou@biol.uoa.gr',
  'echavdoula@gmail.com',
  'geo.papafotiou@gmail.com',
  'pmelissa@bioacademy.gr',
  'akafalidou@bioacademy.gr',
  'npaschal@bioacademy.gr',
  'ipateras@med.uoa.gr',
  'mathan@bioacademy.gr'],
 ['yushanqiu2526374@163.com'],
 ['o.scott@auckland.ac.nz'],
 ['kathleen.frazer@ucd.ie'],
 ['chu-mei.chang@imperial.ac.uk'],
 ['monicake@hawaii.edu',
  'kristimh@hawaii.edu',
  'taflague@triton.uog.edu',
  'bernice7@hawaii.edu',
  'kaholoku@hawaii.edu'],
 ['zhanglei6@hrbmu.edu.cn']]

In [17]:
# pip3 install python-whois
import whois

def is_registered(domain_name):
    try:
        w = whois.whois(domain_name)
    except Exception:
        return False
    else:
        return bool(w.domain_name)

domain_name = "med.toho‑u.ac.jp"
if is_registered(domain_name):
    whois_info = whois.whois(domain_name)
    print(whois_info)
    
print(is_registered(domain_name))

False


In [18]:
# First step is to import the package
import whois21

query = 'med.toho‑u.ac.jp'

# Second step is to create an instance of the WHOIS class
whois = whois21.WHOIS(query)

# Third step is to check if the operation was successful
if not whois.success:
    print(whois.error)
    exit()

# And basically you are done!
# Now you can print the results
import log21  # I use log21 to print the results in a cool way 8D

# Print the results in a nice way
# PPrint the dictionary
log21.pprint(whois.whois_data)
# Tree-Print the dictionary
log21.tree_print(whois.whois_data)

# Or you can print the results in as raw text
print(whois.raw.decode('utf-8'))

# Or you can access each part of the results individually
print(f'Creation date   : {whois.creation_date}')
print(f'Expiration date : {whois.expires_date}')
print(f'Updated date    : {whois.updated_date}')

{'参考': 'IPアドレスのWHOISサーバ',
 '詳しくは HTTPS': '//jprs.jp/about/dom-search/jprs-whois/ を参照してください。'}
─┬ root
 ├───┬ 詳しくは HTTPS
 │   └──── //jprs.jp/about/dom-search/jprs-whois/ を参照してください。
 └───┬ 参考
     └──── IPアドレスのWHOISサーバ



[ JPRS database provides information on network administration. Its use is    ]
[ restricted to network administration purposes. For further information,     ]
[ use 'whois -h whois.jprs.jp help'. To suppress Japanese output, add'/e'     ]
[ at the end of command, e.g. 'whois -h whois.jprs.jp xxx/e'.                 ]

No match!!

JPRS WHOISは、JPRSが管理している以下のドメイン名に関する情報を確認でき
るサービスです。
    ・登録されているJPドメイン名
    ・JPRSを管理レジストラとするgTLD等ドメイン名
詳しくは https://jprs.jp/about/dom-search/jprs-whois/ を参照してください。

参考: IPアドレスのWHOISサーバ
   ・JPNIC WHOIS(whois.nic.ad.jp)
   ・APNIC WHOIS(whois.apnic.net)
   ・ARIN WHOIS(whois.arin.net)
   ・RIPE WHOIS(whois.ripe.net)
   ・LACNIC WHOIS(whois.lacnic.net)
   ・AfriNIC WHOIS(whois.afrinic.net)


Creation date   : 
Expiration date : 
Updated date    : 


In [19]:
import whois11

print(whois11.whois('med.toho‑u.ac.jp'))

[ JPRS database provides information on network administration. Its use is    ]
[ restricted to network administration purposes. For further information,     ]
[ use 'whois -h whois.jprs.jp help'. To suppress Japanese output, add'/e'     ]
[ at the end of command, e.g. 'whois -h whois.jprs.jp xxx/e'.                 ]

No match!!

JPRS WHOISは、JPRSが管理している以下のドメイン名に関する情報を確認でき
るサービスです。
    ・登録されているJPドメイン名
    ・JPRSを管理レジストラとするgTLD等ドメイン名
詳しくは https://jprs.jp/about/dom-search/jprs-whois/ を参照してください。

参考: IPアドレスのWHOISサーバ
   ・JPNIC WHOIS(whois.nic.ad.jp)
   ・APNIC WHOIS(whois.apnic.net)
   ・ARIN WHOIS(whois.arin.net)
   ・RIPE WHOIS(whois.ripe.net)
   ・LACNIC WHOIS(whois.lacnic.net)
   ・AfriNIC WHOIS(whois.afrinic.net)




In [20]:
import pdftotext
import io
from urllib.request import (
    Request,
    urlopen,
)
def file_download(url):
    req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    pdf_pages = pdftotext.PDF(io.BytesIO(urlopen(req).read()))
    return pdf_pages
    
def text_from_pdf_bytes(pdf_bytes):
    pdf_text = pdftotext.PDF(io.BytesIO(urlopen(req).read()))
    return "\n\n".join(pdf_text)

def head_extract(pdf_bytes):
    text = text_from_pdf_file(path)
    abstract_pos = re.search(r"[aA]bstract", text)
    return text[: abstract_pos.start(0) if abstract_pos else 10000]

In [21]:
pdf_pages = file_download("http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9983066&blobtype=pdf")

In [22]:
len(pdf_pages)

8

In [23]:
pdf_pages[0]

'                                                 MEDICINE INTERNATIONAL 3: 11, 2023\n           The combination of positive anti‑WDR1 antibodies with\n       negative anti‑CFL1 antibodies in serum is a poor prognostic\n                      factor for patients with esophageal carcinoma\n          MASAAKI ITO1, SATOSHI YAJIMA2, TAKASHI SUZUKI2, YOKO OSHIMA2, TATSUKI NANAMI2,\n      MAKOTO SUMAZAKI2, FUMIAKI SHIRATORI2, HAO WANG3,4, LIUBING HU4, HIROTAKA TAKIZAWA5,\n                  SHU‑YANG LI6, YASUO IWADATE6, TAKAKI HIWASA1,6 and HIDEAKI SHIMADA1,2\n1\n  Department of Clinical Oncology, Toho University Graduate School of Medicine; 2Department of Gastroenterological Surgery,\n  Toho University School of Medicine, Tokyo 143‑8541, Japan; 3Stroke Center, The First Affiliated Hospital, Jinan University;\n    4\n      Department of Anesthesiology, Stroke Center, The First Affiliated Hospital and Health Science Center, Jinan University,\n        Guangzhou, Guangdong 510630, P.R. China; 5Po

In [24]:
from googlesearch import search 

list_of_queries = ["med.toho‑u.ac.jp"]
results = []

for query in list_of_queries:
    results.extend(list(search(query, num_results=10)))

print(results)

['https://www.toho-u.ac.jp/med/', 'https://search.ebscohost.com/login.aspx?direct=true&profile=ehost&scope=site&authtype=crawler&jrnl=14777819&AN=132477015&h=olj3V%2BBJE0BYK45ZMWkpuUoibzH1V%2FC67AuiTULKlYJSl%2FNxaFT7TiGamATmdbzft2HM%2BHw54uJ1fLEZYHlgyw%3D%3D&crl=c', 'https://www.residentnavi.com/hospitals/1344', 'https://www.chusho.meti.go.jp/sapoin/index.php/application/research/detail/206', 'https://tohobiochemi.jp/member/index.html', 'https://www.mhlw.go.jp/shingi/2009/04/dl/s0423-7d_0002.pdf', 'https://www.toho-u-kyumei99.jp/recruit/', 'https://pubmed.ncbi.nlm.nih.gov/33214049/', 'https://hiv-hospital.jp/list/K117/', 'http://www.toho-kai.jp/', 'https://www.instagram.com/toho_med/']
